# DataGenerator - Progetto Basi di Dati - 2022/2023 - "OrtiScolastici"

Questo notebook contiene il codice per la generazione dei dati fittizi da inserire nel database.

## Import delle librerie

In [8]:
%pip install Faker
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Librerie per la generazione dei dati

In [9]:
import random
from faker import Faker
import psycopg2
import re
import os

## Variabili globali

Per effetturre una connessione corretta al proprio database, è necessario modificare le variabili nella funzione psycopg2.connect(). 

In [10]:
# Connessione al database PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="ProgettoLarge",
    user="postgres",
    password="andrea"
)

In [11]:
# Creazione del cursore
cursor = conn.cursor()

# Creazione dell'istanza di Faker per la generazione di dati casuali
fake = Faker('it_IT')

# Eliminazione del file di popolamento se esiste
file_path = r'..\[ Parte III.A ] PopolamentoLarge.sql'
if os.path.exists(file_path):
    os.remove(file_path)

# Variabili globali per le chiavi esterne
dirigenti = []
scuole = []

# Funzione per la gestione delle query
def execute_query(query, write_to_file=True):
    cursor.execute(query)
    conn.commit()

    if write_to_file:
        with open(file_path, 'a') as f:
            f.write(query + '\n')

## Settaggio del percorso del database

In [12]:
execute_query("SET search_path TO 'OrtiScolastici'")
execute_query("SET datestyle TO 'MDY'")
execute_query("SET timezone TO 'GMT'")

## Inserimento di dati nella tabella Persona

In [13]:
def insert_persona():
    for _ in range(100):
        email = fake.email()
        nome = fake.first_name()
        cognome = fake.last_name()
        telefono = fake.random_number(digits=10)
        rilevatore_esterno = random.choice([True, False])

        query = f"INSERT INTO Persona (Email, Nome, Cognome, Telefono, RilevatoreEsterno) " \
                f"VALUES ('{email}', '{nome}', '{cognome}', '{telefono}', {rilevatore_esterno});"
        cursor.execute(query)

## Inserimento di dati nella tabella Scuola

In [14]:
def insert_scuola():
    for _ in range(10):
        cod_meccanografico = fake.random_number(digits=10)
        nome_scuola = fake.company()
        ciclo_istruzione = random.randint(1, 2)
        comune = re.sub(r'[^\w\s]', '', fake.city())
        provincia = fake.state()
        collabora = random.choice([True, False])
        finanziamento = fake.word()

        dirigenti_stringa = ', '.join(["'" + elemento + "'" for elemento in dirigenti])

        query = f"SELECT Email FROM Persona WHERE RilevatoreEsterno = False" 
        query += f" AND Email NOT IN ({dirigenti_stringa});" if dirigenti_stringa != "" else f";"
        cursor.execute(query)
        rows = cursor.fetchall()

        dirigente = rows[random.randint(0, len(rows) - 1)][0]
        referente = dirigente if finanziamento != "" else 'NULL'

        dirigenti.append(dirigente)

        query = f"INSERT INTO Scuola (Cod_Meccanografico, NomeScuola, CicloIstruzione, Comune, Provincia, Collabora, " \
                f"Finanziamento, Dirigente, Referente) " \
                f"VALUES ('{cod_meccanografico}', '{nome_scuola}', {ciclo_istruzione}, '{comune}', '{provincia}', " \
                f"{collabora}, '{finanziamento}', '{dirigente}', '{referente}');"
        execute_query(query)

## Inserimento di dati nella tabella Classe

In [15]:
def insert_classe():
    # Selezione di una scuola a caso
    query = f"SELECT Cod_Meccanografico FROM Scuola;"
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        scuole.append(row[0])

    scuola = None
    
    for i in range(0, 19):
        id_classe = fake.random_number(digits=10)
        sezione = fake.random_letter()
        ordine = random.randint(1, 2)
        
        if i < 2:
            scuola = scuole[0]  # Prima scuola
        elif i < len(scuole):
            scuola = scuole[(i // 2) * 2]  # Scuola in base all'indice
        else:
            scuola = scuole[len(scuole) - 1]
        tipo_scuola = fake.word()
        
        # Query per escludere i dirigenti dalle classi
        dirigenti_stringa = ', '.join(["'" + elemento + "'" for elemento in dirigenti])

        query = f"SELECT Email FROM Persona WHERE RilevatoreEsterno = False" 
        query += f" AND Email NOT IN ({dirigenti_stringa});" if dirigenti_stringa != "" else f";"
        cursor.execute(query)
        rows = cursor.fetchall()

        docente = rows[random.randint(0, len(rows) - 1)][0]

        query = f"INSERT INTO Classe (IdClasse, Sezione, Scuola, Ordine, TipoScuola, Docente) " \
                f"VALUES ('{id_classe}', '{sezione}', '{scuola}', {ordine}, '{tipo_scuola}', '{docente}');"
        execute_query(query)

## Inserimento di dati nella tabella Specie

In [16]:
def insert_specie():
    substrato_choices = ['TerriccioRinvaso', 'SuoloPreEsistente']

    for _ in range(50):
        nome_scientifico = fake.unique.word()
        substrato = random.choice(substrato_choices)

        query = f"INSERT INTO Specie (NomeScientifico, Substrato) VALUES ('{nome_scientifico}', '{substrato}');"
        execute_query(query)

## Inserimento di dati nella tabella Orto

In [17]:
def insert_orto():
    query = "SELECT Cod_Meccanografico FROM Scuola;"
    cursor.execute(query)
    scuola_ids = [row[0] for row in cursor.fetchall()]

    query = "SELECT NomeScientifico FROM Specie;"
    cursor.execute(query)
    specie_names = [row[0] for row in cursor.fetchall()]

    for _ in range(30):
        id_orto = fake.random_number(digits=10)
        nome_orto = fake.word()
        latitudine = fake.latitude()
        longitudine = fake.longitude()
        superficie = random.uniform(1, 100)
        posizione = random.choice(['Vaso', 'Terra'])
        condizione_ambientale = random.choice(['Pulito', 'Inquinato'])
        specie = random.choice(specie_names)
        scuola = random.choice(scuola_ids)

        query = f"INSERT INTO Orto (IdOrto, NomeOrto, Latitudine, Longitudine, Superficie, Posizione, " \
                f"CondizioneAmbientale, Specie, Scuola) " \
                f"VALUES ('{id_orto}', '{nome_orto}', '{latitudine}', '{longitudine}', '{superficie}', " \
                f"'{posizione}', '{condizione_ambientale}', '{specie}', '{scuola}');"
        execute_query(query)

## Inserimento di dati nella tabella Pianta

In [18]:
def insert_pianta():
    query = "SELECT IdClasse FROM Classe;"
    cursor.execute(query)
    class_ids = [row[0] for row in cursor.fetchall()]

    query = "SELECT NomeScientifico FROM Specie;"
    cursor.execute(query)
    specie_names = [row[0] for row in cursor.fetchall()]

    for _ in range(100):
        numero_replica = fake.random_number(digits=10)
        nome_comune = fake.word()
        scopo = random.choice(['Fitobotanica', 'Biomonitoraggio'])
        data_messa_dimora = fake.date()

        specie = random.choice(specie_names)
        classe = random.choice(class_ids)

        query = f"INSERT INTO Pianta (NumeroReplica, NomeComune, Scopo, DataMessaDimora, Specie, Classe) " \
                f"VALUES ('{numero_replica}', '{nome_comune}', '{scopo}', '{data_messa_dimora}', " \
                f"'{specie}', '{classe}');"
        execute_query(query)

## Inserimento di dati nella tabella Esposizione

In [19]:
def insert_esposizione():
    query = "SELECT NumeroReplica, NomeComune FROM Pianta;"
    cursor.execute(query)
    pianta_data = cursor.fetchall()

    for pianta in pianta_data:
        numero_replica, nome_comune = pianta
        tipo_esposizione = random.choice(['Sole', 'Mezzombra', 'Ombra'])

        query = f"INSERT INTO Esposizione (NumeroReplica, NomeComune, TipoEsposizione) " \
                f"VALUES ('{numero_replica}', '{nome_comune}', '{tipo_esposizione}');"
        execute_query(query)

## Inserimento di dati nella tabella Gruppo

In [20]:
def insert_gruppo():
    tipo_gruppo = ['Controllo', 'Monitoraggio']

    query = "SELECT NumeroReplica, NomeComune FROM Pianta;"
    cursor.execute(query)
    pianta_data = cursor.fetchall()

    for pianta in pianta_data:
        numero_replica, nome_comune = pianta
        tipo = random.choice(tipo_gruppo)
        id_gruppo = fake.random_int(min=1, max=100)

        query = f"INSERT INTO Gruppo (IdGruppo, TipoGruppo, NumeroReplica, NomeComune) " \
                f"VALUES ({id_gruppo}, '{tipo}', '{numero_replica}', '{nome_comune}');"
        execute_query(query)

## Inserimento di dati nella tabella Sensore

In [21]:
def insert_sensore():
    tipo_sensore = ['Arduino', 'Sensore']
    tipo_acquisizione = ['Arduino', 'App']

    for i in range(1, 6):
        tipo = random.choice(tipo_sensore)
        acquisizione = random.choice(tipo_acquisizione)

        query = f"INSERT INTO Sensore (IdSensore, TipoSensore, TipoAcquisizione) " \
                f"VALUES ({i}, '{tipo}', '{acquisizione}');"
        execute_query(query)

## Inserimento di dati nella tabella Rilevazione

In [22]:
def insert_rilevazione():
    query = "SELECT IdSensore FROM Sensore;"
    cursor.execute(query)
    sensore_ids = [row[0] for row in cursor.fetchall()]

    query = "SELECT NumeroReplica, NomeComune FROM Pianta;"
    cursor.execute(query)
    pianta_data = cursor.fetchall()

    index = 1
    for pianta in pianta_data:
        numero_replica, nome_comune = pianta
        data_ora_rilevazione = fake.date_time_this_decade()
        data_ora_inserimento = fake.date_time_between(start_date=data_ora_rilevazione)

        sensore = random.choice(sensore_ids)

        query = f"INSERT INTO Rilevazione (IdRilevazione, NumeroReplica, NomeComune, DataOraRilevazione, DataOraInserimento, " \
                f"Sensore) " \
                f"VALUES ({index}, '{numero_replica}', '{nome_comune}', '{data_ora_rilevazione}', '{data_ora_inserimento}', " \
                f"'{sensore}');"
        execute_query(query)
        index += 1

## Inserimento di dati nella tabella Dati

In [23]:
def insert_dati():
    query = "SELECT IdRilevazione FROM Rilevazione;"
    cursor.execute(query)
    rilevazione_ids = [row[0] for row in cursor.fetchall()]

    for rilevazione in rilevazione_ids:
        temperatura = random.uniform(10, 30)
        umidita = random.uniform(30, 80)
        ph = random.uniform(5, 8)
        foglie_danneggiate = random.randint(0, 10)
        superficie_foglie_danneggiate = random.uniform(0, 100)
        fiori = random.randint(0, 20)
        frutti = random.randint(0, 50)
        altezza_pianta = random.uniform(10, 200)
        lunghezza_radice = random.uniform(10, 200)

        query = f"INSERT INTO Dati (Rilevazione, Temperatura, Umidita, Ph, FoglieDanneggiate, " \
                f"SuperficieFoglieDanneggiate, Fiori, Frutti, AltezzaPianta, LunghezzaRadice) " \
                f"VALUES ('{rilevazione}', {temperatura}, {umidita}, {ph}, {foglie_danneggiate}, " \
                f"{superficie_foglie_danneggiate}, {fiori}, {frutti}, {altezza_pianta}, {lunghezza_radice});"
        execute_query(query)

## Inserimento di dati nella tabella Responsabile

In [24]:
def insert_responsabile():
    query = "SELECT IdRilevazione FROM Rilevazione;"
    cursor.execute(query)
    rilevazione_ids = [row[0] for row in cursor.fetchall()]

    query = "SELECT Email FROM Persona;"
    cursor.execute(query)
    persona_emails = [row[0] for row in cursor.fetchall()]

    query = "SELECT IdClasse FROM Classe;"
    cursor.execute(query)
    classe_ids = [row[0] for row in cursor.fetchall()]

    for rilevazione in rilevazione_ids:
        
        index = random.randint(0, 3)

        if index == 0:
            inserimento_persona = random.choice(persona_emails)
            query = f"INSERT INTO Responsabile (Rilevazione, InserimentoPersona, RilevatorePersona, " \
                f"InserimentoClasse, RilevatoreClasse) " \
                f"VALUES ('{rilevazione}', '{inserimento_persona}', NULL, " \
                f"NULL, NULL);"
        elif index == 1:
            rilevatore_persona = random.choice(persona_emails)
            inserimento_classe = random.choice(classe_ids)
            query = f"INSERT INTO Responsabile (Rilevazione, InserimentoPersona, RilevatorePersona, " \
                f"InserimentoClasse, RilevatoreClasse) " \
                f"VALUES ('{rilevazione}', NULL, '{rilevatore_persona}', " \
                f"'{inserimento_classe}', NULL);"
        elif index == 2:
            inserimento_persona = random.choice(persona_emails)
            rilevatore_classe = random.choice(classe_ids)
            query = f"INSERT INTO Responsabile (Rilevazione, InserimentoPersona, RilevatorePersona, " \
                f"InserimentoClasse, RilevatoreClasse) " \
                f"VALUES ('{rilevazione}', '{inserimento_persona}', NULL, " \
                f"NULL, '{rilevatore_classe}');"
        else:
            inserimento_persona = None
            rilevatore_persona = None
            inserimento_classe = random.choice(classe_ids)
            rilevatore_classe = None
            query = f"INSERT INTO Responsabile (Rilevazione, InserimentoPersona, RilevatorePersona, " \
                f"InserimentoClasse, RilevatoreClasse) " \
                f"VALUES ('{rilevazione}', NULL, NULL, " \
                f"{inserimento_classe}, NULL);"

        execute_query(query)

In [25]:
insert_persona()

In [26]:
insert_scuola()

In [27]:
insert_classe()

In [28]:
insert_specie()

In [29]:
insert_orto()

In [30]:
insert_pianta()

In [31]:
insert_esposizione()

In [32]:
insert_gruppo()

In [33]:
insert_sensore()

In [34]:
insert_rilevazione()

In [35]:
insert_dati()

In [36]:
insert_responsabile()

In [37]:
# Close the database connection
cursor.close()
conn.close()